## **Applying Advanced Transformations (Core)**

**Author: P. Slabber**

**The Task**

Your task is two-fold:

### **I. Clean the files and combine them into one final DataFrame.**

- This dataframe should have the following columns:
   - Hero (Just the name of the Hero)
   - Publisher
   - Gender
   - Eye color
   - Race
   - Hair color
   - Height (numeric)
   - Skin color
   - Alignment
   - Weight (numeric)
   - Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
     - Agility
     - Flight
     - Superspeed
     - etc.
 
Hint: There is a space in "100 keach publisher?


### **II. Use your combined DataFrame to answer the following questions.**

 - Compare the average weight of super powers who have Super Speed to those who do not.
 - What is the average height of heroes for each publisher?

### **Loading the Data**

In [3]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

### **Load Super Hero Info data**

In [14]:
df_info = pd.read_csv('/Users/tspiet/Documents/GitHub/JSON_API/Data/superhero_info.csv')
df_info.head()


,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


### **I. Clean the files and combine them into one final DataFrame.**

### **Split the Hero and Publisher**

In [15]:
## adding expand=True
df_info['Hero|Publisher'].str.split('|',expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [16]:
## save the 2 new columns into the dataframe
pd.set_option('display.max_columns', None)
df_info[['Hero', 'Publisher']] = df_info['Hero|Publisher'].str.split('|',expand=True)
df_info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [17]:
## drop the original column 
pd.set_option('display.max_columns', None)
df_info = df_info.drop(columns=['Hero|Publisher'])
df_info.head(2)


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [18]:
### **Split Measurement into Height and Weight**

In [19]:
## adding expand=True
df_info['Measurements'].str.split(',',expand=True)


,0,1
0,{'Height': '203.0 cm','Weight': '441.0 kg'}
1,{'Height': '191.0 cm','Weight': '65.0 kg'}
2,{'Height': '185.0 cm','Weight': '90.0 kg'}
3,{'Height': '203.0 cm','Weight': '441.0 kg'}
4,{'Height': '193.0 cm','Weight': '122.0 kg'}
...,...,...
458,{'Height': '183.0 cm','Weight': '83.0 kg'}
459,{'Height': '165.0 cm','Weight': '52.0 kg'}
460,{'Height': '66.0 cm','Weight': '17.0 kg'}
461,{'Height': '170.0 cm','Weight': '57.0 kg'}


In [20]:
## save the 2 new columns into the dataframe
pd.set_option('display.max_columns', None)
df_info[['Height', 'Weight']] = df_info['Measurements'].str.split(',',expand=True)
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,{'Height': '203.0 cm','Weight': '441.0 kg'}
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,{'Height': '191.0 cm','Weight': '65.0 kg'}


In [21]:
## drop the original column 
df_info = df_info.drop(columns=['Measurements'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,{'Height': '203.0 cm','Weight': '441.0 kg'}
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,{'Height': '191.0 cm','Weight': '65.0 kg'}


In [22]:
# Remove Unnecessary Characters from the Height column
# Make a list of all characters to replace
to_replace = ['{', "'", 'Height:', ':', 'cm', ' ']
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    df_info['Height'] = df_info['Height'].str.replace(char,'',regex=False)
df_info['Height'].head()


0    203.0
1    191.0
2    185.0
3    203.0
4    193.0
Name: Height, dtype: object

In [23]:
# Remove Unnecessary Characters from the Weight column
# Make a list of all characters to replace
to_replace = ['}', "'", 'Weight:', ':', 'kg', ' ']
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    df_info['Weight'] = df_info['Weight'].str.replace(char,'',regex=False)
df_info['Weight'].head()


0    441.0
1     65.0
2     90.0
3    441.0
4    122.0
Name: Weight, dtype: object

In [24]:
# Convert string datatype to Numeric datatype
df_info['Height'] = pd.to_numeric(df_info['Height'].str.extract('(\d+\.\d+)')[0])
df_info['Weight'] = pd.to_numeric(df_info['Weight'].str.extract('(\d+\.\d+)')[0])

In [25]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


### **Load Super Powers Data**

In [26]:
df_powers = pd.read_csv('/Users/tspiet/Documents/GitHub/JSON_API/Data/superhero_powers.csv')
df_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [27]:
# Split the comma-separated string into a list of powers
df_powers['Powers'] = df_powers['Powers'].str.split(',')

# Check the results
df_powers['Powers'].head()

0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: Powers, dtype: object

In [28]:
df_powers['Powers'].value_counts()

[Intelligence]                                                                                                                                                                                                                                                                          8
[Durability, Super Strength]                                                                                                                                                                                                                                                            5
[Agility, Stealth, Marksmanship, Weapons Master, Stamina]                                                                                                                                                                                                                               4
[Marksmanship]                                                                                                                                            

In [29]:
exploded = df_powers.explode('Powers')
result_df = exploded[['hero_names', 'Powers']]
result_df.head(5)

,hero_names,Powers
0,3-D Man,Agility
0,3-D Man,Super Strength
0,3-D Man,Stamina
0,3-D Man,Super Speed
1,A-Bomb,Accelerated Healing


In [30]:
# Get unique values from the 'Powers' column
cols_to_make = exploded['Powers'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [32]:
for col in cols_to_make:
    result_df[col] = result_df['Powers'].str.contains(col)
result_df.head()

,hero_names,Powers,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,Cold Resistance,Underwater breathing,Marksmanship,Weapons Master,Intelligence,Telepathy,Immortality,Reflexes,Enhanced Sight,Sub-Mariner,Lantern Power Ring,Invulnerability,Animation,Super Breath,Dimensional Awareness,Flight,Size Changing,Teleportation,Magic,Dimensional Travel,Molecular Manipulation,Energy Manipulation,Power Cosmic,Energy Absorption,Elemental Transmogrification,Fire Resistance,Natural Armor,Heat Resistance,Matter Absorption,Regeneration,Stealth,Power Suit,Energy Blasts,Energy Beams,Heat Generation,Danger Sense,Phasing,Force Fields,Hypnokinesis,Invisibility,Enhanced Senses,Jump,Shapeshifting,Elasticity,Substance Secretion,Enhanced Hearing,Adaptation,Gliding,Natural Weapons,Wallcrawling,Vision - Thermal,Power Augmentation,Cryokinesis,Duplication,Weapon-based Powers,Fire Control,Enhanced Memory,Energy Constructs,Power Absorption,Time Travel,Enhanced Smell,Water Control,Vision - Telescopic,Magnetism,Vision - Microscopic,Vision - Night,Vision - Heat,Vision - X-Ray,Psionic Powers,Toxin and Disease Resistance,Animal Attributes,Animal Oriented Powers,Toxin and Disease Control,Empathy,Animal Control,Darkforce Manipulation,Time Manipulation,Reality Warping,Web Creation,Symbiote Costume,Telekinesis,Levitation,Biokinesis,Sonar,Enhanced Touch,Seismic Power,Energy Armor,Omnilingualism,Radiation Immunity,Density Control,Light Control,Illumination,Molecular Combustion,Astral Travel,Precognition,Intangibility,Illusions,Audio Control,Sonic Scream,Peak Human Condition,Vision - Infrared,Dexterity,Omnitrix,Omnipotent,Possession,Energy Resistance,Vision - Cryo,Mind Control,Mind Blast,Clairvoyance,Probability Manipulation,Death Touch,Electrokinesis,Gravity Control,Technopath/Cyberpath,Radar Sense,Astral Projection,Cloaking,Telepathy Resistance,Mind Control Resistance,Hyperkinesis,Spatial Awareness,Radiation Control,Radiation Absorption,Magic Resistance,Element Control,Plant Control,Terrakinesis,Power Sense,Wind Control,Qwardian Power Ring,Echolocation,Resurrection,The Force,Summoning,Banish,Molecular Dissipation,Electrical Transport,Speed Force,Grim Reaping,Portal Creation,Vitakinesis,Projection,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,Agility,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
0,3-D Man,Super Strength,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [33]:
# Rename the 'hero_names' column to 'Hero'
result_df.rename(columns={'hero_names': 'Hero'}, inplace=True)

In [34]:
# Merge the two DataFrames on 'Hero'
merged_df = pd.merge(df_info, result_df, on='Hero')

In [35]:
merged_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Powers,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,Cold Resistance,Underwater breathing,Marksmanship,Weapons Master,Intelligence,Telepathy,Immortality,Reflexes,Enhanced Sight,Sub-Mariner,Lantern Power Ring,Invulnerability,Animation,Super Breath,Dimensional Awareness,Flight,Size Changing,Teleportation,Magic,Dimensional Travel,Molecular Manipulation,Energy Manipulation,Power Cosmic,Energy Absorption,Elemental Transmogrification,Fire Resistance,Natural Armor,Heat Resistance,Matter Absorption,Regeneration,Stealth,Power Suit,Energy Blasts,Energy Beams,Heat Generation,Danger Sense,Phasing,Force Fields,Hypnokinesis,Invisibility,Enhanced Senses,Jump,Shapeshifting,Elasticity,Substance Secretion,Enhanced Hearing,Adaptation,Gliding,Natural Weapons,Wallcrawling,Vision - Thermal,Power Augmentation,Cryokinesis,Duplication,Weapon-based Powers,Fire Control,Enhanced Memory,Energy Constructs,Power Absorption,Time Travel,Enhanced Smell,Water Control,Vision - Telescopic,Magnetism,Vision - Microscopic,Vision - Night,Vision - Heat,Vision - X-Ray,Psionic Powers,Toxin and Disease Resistance,Animal Attributes,Animal Oriented Powers,Toxin and Disease Control,Empathy,Animal Control,Darkforce Manipulation,Time Manipulation,Reality Warping,Web Creation,Symbiote Costume,Telekinesis,Levitation,Biokinesis,Sonar,Enhanced Touch,Seismic Power,Energy Armor,Omnilingualism,Radiation Immunity,Density Control,Light Control,Illumination,Molecular Combustion,Astral Travel,Precognition,Intangibility,Illusions,Audio Control,Sonic Scream,Peak Human Condition,Vision - Infrared,Dexterity,Omnitrix,Omnipotent,Possession,Energy Resistance,Vision - Cryo,Mind Control,Mind Blast,Clairvoyance,Probability Manipulation,Death Touch,Electrokinesis,Gravity Control,Technopath/Cyberpath,Radar Sense,Astral Projection,Cloaking,Telepathy Resistance,Mind Control Resistance,Hyperkinesis,Spatial Awareness,Radiation Control,Radiation Absorption,Magic Resistance,Element Control,Plant Control,Terrakinesis,Power Sense,Wind Control,Qwardian Power Ring,Echolocation,Resurrection,The Force,Summoning,Banish,Molecular Dissipation,Electrical Transport,Speed Force,Grim Reaping,Portal Creation,Vitakinesis,Projection,Insanity,Photographic Reflexes,Anti-Gravity,Thirstokinesis,Power Nullifier,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Accelerated Healing,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,Durability,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

### **II. Use your combined DataFrame to answer the following questions.**

 - Compare the average weight of super powers who have Super Speed to those who do not.
 - What is the average height of heroes for each publisher?

In [37]:
# Calculate the average weight of heroes with Super Speed
average_weight_with_speed = merged_df[merged_df['Powers'] == 'Super Speed']['Weight'].mean()

# Calculate the average weight of heroes without Super Speed
average_weight_without_speed = merged_df[merged_df['Powers'] != 'Super Speed']['Weight'].mean()

print(f'Average weight of superheroes with Super Speed: {average_weight_with_speed:.2f}')
print(f'Average weight of superheroes without Super Speed: {average_weight_without_speed:.2f}')


Average weight of superheroes with Super Speed: 129.40
Average weight of superheroes without Super Speed: 126.36


In [38]:
# Calculate average height for each publisher
average_height_by_publisher = merged_df.groupby('Publisher')['Height'].mean()

print(average_height_by_publisher)

Publisher
DC Comics            187.582386
Dark Horse Comics    181.486667
George Lucas         154.692308
Image Comics         211.000000
Marvel Comics        201.015327
Shueisha             171.012658
Star Trek            182.500000
Team Epic TV         181.692308
Unknown              178.000000
Name: Height, dtype: float64
